In [42]:
#default_exp tests

In [43]:
#hide
%load_ext autoreload
%autoreload 2    
%load_ext line_profiler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [44]:
#export
from pathlib import Path
from datetime import timedelta, datetime
import pandas as pd
from tqdm.notebook import tqdm
from rfpy.main import parse_input
from rfpy.parser import get_files
from rfpy.blocks import *
from rfpy.utils import binary_file_generator
from nbdev.showdoc import *
from fastcore.xtras import parallel
from fastcore.foundation import L
from fastcore.test import *

Com o objetivo de estudar o código fui reescrevendo-o. Tentar reescrever funções é um bom exercício para entender e internalizar a implementação utilizada. Assim foi possível identificar várias oportunidades para simplesmente refatorar e simplificar o código. 

Utilizarei várias funcinalidades da biblioteca [fastcore](https://fastcore.fast.ai/basics.html), esta  extende de maneira muito eficiente várias funcionalidades básicas do python e assim não precisamos _reiventar a roda_.

### `config.tradutor.yaml` 

Como só temos 2 parâmetros, `entrada` e `saída` escolhi não utilizar um arquivo de configuração por não achar ser necessário. Outro benefício dessa abordagem é utilizar as variáveis que antes estavam no arquivo de configuração como parâmetros de entrada numa linha de comando.

Como não utilizei um arquivo de configuração não é necessário chamar a função `busca_arquivos` do arquivo `helper.py`

In [45]:
VERBOSE = True
entrada = Path(r'D:\OneDrive - ANATEL\Backup_Rfeye_SP\RPO\PMEC2020\Ribeirao_Preto_SP\SLMA')
saida = Path(r'C:\Users\rsilva\Downloads')

In [46]:
show_doc(get_files)

<h4 id="get_files" class="doc_header"><code>get_files</code><a href="https://github.com/ronaldokun/rfpy/tree/master/rfpy/parser.py#L17" class="source_link" style="float:right">[source]</a></h4>

> <code>get_files</code>(**`path`**, **`extensions`**=*`None`*, **`recurse`**=*`True`*, **`folders`**=*`None`*, **`followlinks`**=*`True`*)

Get all the files in `path` with optional `extensions`, optionally with `recurse`, only in `folders`, if specified.

In [47]:
arquivos = get_files(entrada) ; arquivos

(#255) [Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200317_232233.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225525.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225535.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225556.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200318_225615.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200410_183001.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/rfeye002304_SLMA_bimestral_occ15min_200423_162327.bin'),Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/

O Objeto retornado `L` é uma extensão da lista python com funcionalidades adicionais, uma delas como  podemos ver é que a representaçõa da lista impressa vem com o número de arquivos da lista. 
Temos 255 arquivos bin na pasta entrada. Podemos filtrar por pasta também

In [48]:
arquivos_bin = get_files(entrada, extensions=['.bin'], folders='tmp')

In [49]:
arquivos_bin

(#1) [Path('D:/OneDrive - ANATEL/Backup_Rfeye_SP/RPO/PMEC2020/Ribeirao_Preto_SP/SLMA/tmp/rfeye002304_SLMA_bimestral_PEAK_200829_234902.bin')]

O Código executado em `tradutor_RFEYE.py` está totalmente contido de forma mais geral em `main.py`. Chamei de main justamente porque esse será o arquivo base que chama outros arquivos e funções.

In [50]:
LINHA_A = [[1,  '',  '',  '',  datetime(2020, 8, 29, 23, 50, 1, 59268),  38373,  108,
          137, 14848, 'peak', 1, '']]
LINHA_B = [1, 108.0, -75.0]

In [51]:
save_file_a = saida / 'file_a.csv'
save_file_b = saida / 'file_b.csv'

A = ['Block_Number', 'Latitude', 'Longitude', 'Altitude', 'Initial_Time',
     'Sample_Duration', 'Start_Frequency', 'Stop_Frequency', 'Vector_Length', 
     'Trace_Type', 'Antenna_Type', 'Equipement_ID']

B = ['Block_Number', 'Frequency(HZ)', 'Nivel(dBm)']

file_a = [] #pd.DataFrame(columns=A)
file_b = [] #pd.DataFrame(columns=B, dtype='float16')

block_apply = {}



lat = 0
long = 0
alt = 0
conta_bloco = 0  # contador de blocos processados

for file in arquivos_bin:
#     data = L(binary_file_generator(file))
#     tipos_bloco = data.map(Block).attrgot('block_type')    
    for i, data in enumerate(tqdm(binary_file_generator(file))):
        bloco = Block(data)
        hostname = ''
        latitude = ''
        longitude = ''
        altitude = ''
        if bloco.block_type == 21:
            hostname = DataType21(data).hostname
        if bloco.block_type == 40:  # GPS Data
            gps = DataType40(data)
            latitude = gps.latitude
            longitude = gps.longitude
            altitude = gps.altitude
        if bloco.block_type == 63:
            break
            conta_bloco += 1  # contador de blocos processados
            espectro = DataType63(data)
            start_datetime_stamp = espectro.datetime_stamp
            sample_duration = espectro.spent_time_microsecs
            start_frequency = espectro.start_mega
            stop_frequency = espectro.stop_mega
            antena_id = espectro.ant_id
            trace_type = espectro.processing
            rbw = espectro.rbw
            vector_length = espectro.data_points
            file_a.extend([[conta_bloco, latitude, longitude, altitude,start_datetime_stamp,
                      sample_duration, start_frequency, stop_frequency,vector_length,
                      trace_type, antena_id, hostname]])           
            
            file_b.extend([[conta_bloco, freq, nivel] for freq, nivel in zip(espectro.list_freq_mega, espectro.block_data)])
            #if i > 1000:
            break
            #file_a = file_a.append(pd.DataFrame(linha_a, columns=A))
            #file_b = file_b.append(pd.DataFrame(linha_b, columns=B))

In [ ]:
espectro = DataType63(data)